In [3]:
!pip install imgaug
!pip install terminaltables
!pip install torchsummary
!pip install pycocotools

  Stored in directory: /home/jovyan/.cache/pip/wheels/68/a5/e7/56401832f23d0b2db351c5b682e466cb4841960b086da65e4e
Successfully built pycocotools


In [16]:
from pytorchyolo.utils.datasets import ListDataset
from pytorchyolo.utils.augmentations import AUGMENTATION_TRANSFORMS
from pytorchyolo.utils.utils import to_cpu, load_classes, print_environment_info, provide_determinism, worker_seed_set
from pycocotools.coco import COCO
import os
import shutil
from tqdm import tqdm
import skimage.io as io
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageDraw
from torch.utils.data import DataLoader

In [17]:
def id2name(coco):
    classes=dict()
    for cls in coco.dataset['categories']:
        classes[cls['id']]=cls['name']
    return classes

In [8]:
def _create_data_loader(img_path, batch_size, img_size, n_cpu, multiscale_training=False):
    """Creates a DataLoader for training.

    :param img_path: Path to file containing all paths to training images.
    :type img_path: str
    :param batch_size: Size of each image batch
    :type batch_size: int
    :param img_size: Size of each image dimension for yolo
    :type img_size: int
    :param n_cpu: Number of cpu threads to use during batch generation
    :type n_cpu: int
    :param multiscale_training: Scale images to different sizes randomly
    :type multiscale_training: bool
    :return: Returns DataLoader
    :rtype: DataLoader
    """
    dataset = ListDataset(
        img_path,
        img_size=img_size,
        multiscale=multiscale_training,
        transform=AUGMENTATION_TRANSFORMS)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=n_cpu,
        pin_memory=True,
        collate_fn=dataset.collate_fn,
        worker_init_fn=worker_seed_set)
    return dataloader, dataset

In [19]:
def dataset_generator(server_num, batch_size, n_cpu, multiscale_training=False,img_size=416,iid=True, mode="train",):
    dataDir = '.'
    dataset = mode+'2014'
    annFile='{}/annotations/instances_{}.json'.format(dataDir,dataset)
    classes_names = ['car', 'bicycle', 'person', 'motorcycle', 'bus', 'truck']
    coco = COCO(annFile)
    classes = id2name(coco)
    print(classes)
    classes_ids = coco.getCatIds(catNms=classes_names)
    print(classes_ids)
    if iid is True:
        text= dict()
        for i_s in range(server_num):
            file_out = dataset+'server'+'_'+str(i_s)+'_'+'.txt'
            f = open(file_out,'a')
            for cls in classes_names:
                    #Get ID number of this class
                    cls_id=coco.getCatIds(catNms=[cls])
                    img_ids=coco.getImgIds(catIds=cls_id)
                    total = len(img_ids)
                    per   = int(total/server_num)
                    img_ids_ = img_ids[(i_s)*per:(i_s+1)*per]

                    for imgId in tqdm(img_ids_):
                        img = coco.loadImgs(imgId)[0]
                        filename = img['file_name']
                        string = './images/'+dataset+'/'+filename
                        #print(string)
                        f.write(string)
                        f.write('\n')
            f.close()
            f = open(file_out,"r")
            
            
            writeDir = "./new_"+dataset+'server'+'_'+str(i_s)+'_'+".txt"
            
            outfile=open(writeDir,"w")

            lines_seen = set()  # Build an unordered collection of unique elements.
 
            for line in f:
                line = line.strip('\n')
                if line not in lines_seen:
                    outfile.write(line+ '\n')
                    lines_seen.add(line)
            outfile.close()
            text[i_s] = writeDir 
            
        dataloader_ser={}
        dataset_ser = {}
    
        for i_s in range(server_num):
            img_path = text[i_s]
            dataloader,dataset =  _create_data_loader(img_path, batch_size, img_size, n_cpu, multiscale_training=False)
            dataloader_ser[i_s] = dataloader
            dataset_ser[i_s] = dataset
        print('done')
        
        return dataloader_ser,dataset_ser
            
            
            
            


            

In [21]:
a, b = dataset_generator(5,16,8)

loading annotations into memory...
Done (t=11.33s)
creating index...


  0%|          | 0/9034 [00:00<?, ?it/s]

index created!
{1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'cell pho

100%|██████████| 864/864 [00:00<00:00, 275496.32it/s]


done
